# Chapter 30: AI Integration

Artificial intelligence is transforming modern web applications, enabling dynamic content generation, intelligent chat interfaces, and automated workflows. Next.js provides first-class support for AI integration through the Vercel AI SDK, offering streaming capabilities, tool calling, and framework-agnostic compatibility. Implementing AI features requires careful consideration of streaming architecture, cost management, and privacy safeguards.

By the end of this chapter, you'll master implementing streaming chat interfaces with the AI SDK, building tool-augmented AI agents, optimizing token usage and costs, handling AI errors gracefully, maintaining user privacy with edge AI processing, and architecting real-world AI features for production applications.

## 30.1 Vercel AI SDK Setup

Configure the AI SDK with multiple providers and streaming capabilities.

### Provider Configuration

```typescript
// lib/ai/providers.ts
import { createOpenAI } from '@ai-sdk/openai';
import { createAnthropic } from '@ai-sdk/anthropic';
import { createGoogleGenerativeAI } from '@ai-sdk/google';
import { LanguageModel } from 'ai';

// Provider factory for flexible model selection
export function createModel(provider: string, modelId: string): LanguageModel {
  switch (provider) {
    case 'openai':
      const openai = createOpenAI({
        apiKey: process.env.OPENAI_API_KEY,
        compatibility: 'strict',
      });
      return openai(modelId);
      
    case 'anthropic':
      const anthropic = createAnthropic({
        apiKey: process.env.ANTHROPIC_API_KEY,
      });
      return anthropic(modelId);
      
    case 'google':
      const google = createGoogleGenerativeAI({
        apiKey: process.env.GOOGLE_GENERATIVE_AI_API_KEY,
      });
      return google(modelId);
      
    default:
      throw new Error(`Unknown provider: ${provider}`);
  }
}

// Model selection based on use case
export const models = {
  fast: createModel('openai', 'gpt-4o-mini'),      // For quick responses
  balanced: createModel('openai', 'gpt-4o'),       // For general use
  powerful: createModel('anthropic', 'claude-3-5-sonnet-20241022'), // For complex reasoning
  vision: createModel('openai', 'gpt-4o-vision-preview'), // For image analysis
};
```

### Basic Streaming Chat Route

```typescript
// app/api/chat/route.ts
import { streamText, convertToCoreMessages, tool } from 'ai';
import { z } from 'zod';
import { models } from '@/lib/ai/providers';

export const maxDuration = 30; // Allow 30s for streaming

export async function POST(req: Request) {
  const { messages, model = 'balanced' } = await req.json();

  const result = await streamText({
    model: models[model as keyof typeof models],
    system: `You are a helpful Next.js coding assistant. 
      Provide concise, accurate answers with code examples when relevant.
      Always use TypeScript and App Router patterns.`,
    messages: convertToCoreMessages(messages),
    maxTokens: 2000,
    temperature: 0.7,
    tools: {
      getDocumentation: tool({
        description: 'Get the latest Next.js documentation for a specific topic',
        parameters: z.object({
          topic: z.string().describe('The topic to search for (e.g., "server components", "routing")'),
          version: z.string().optional().describe('Next.js version, defaults to 14'),
        }),
        execute: async ({ topic, version = '14' }) => {
          // Fetch from docs API or search index
          const docs = await fetchDocumentation(topic, version);
          return {
            title: docs.title,
            content: docs.content,
            url: docs.url,
          };
        },
      }),
      searchCodeExamples: tool({
        description: 'Search for relevant code examples in the knowledge base',
        parameters: z.object({
          query: z.string().describe('The search query for code patterns'),
          framework: z.enum(['nextjs', 'react', 'typescript']).default('nextjs'),
        }),
        execute: async ({ query, framework }) => {
          return await searchExamples(query, framework);
        },
      }),
    },
    onFinish: async ({ text, toolCalls, toolResults, finishReason, usage }) => {
      // Log usage for analytics/cost tracking
      console.log('AI Usage:', {
        promptTokens: usage.promptTokens,
        completionTokens: usage.completionTokens,
        totalTokens: usage.totalTokens,
        finishReason,
      });
      
      // Store conversation history if needed
      await saveConversation(messages, text, toolCalls);
    },
  });

  return result.toDataStreamResponse();
}

// Helper functions
async function fetchDocumentation(topic: string, version: string) {
  // Implementation would query your docs or external API
  return {
    title: `Next.js ${version} - ${topic}`,
    content: 'Documentation content...',
    url: `https://nextjs.org/docs/${version}/${topic}`,
  };
}

async function searchExamples(query: string, framework: string) {
  // Search vector database or code repository
  return [
    { title: 'Example 1', code: '...', relevance: 0.95 },
  ];
}

async function saveConversation(messages: any[], response: string, tools: any[]) {
  // Persist to database for context windows or analytics
}
```

## 30.2 AI-Powered UI Components

Build React components that handle streaming responses and tool invocations.

### Streaming Chat Interface

```typescript
// components/ai/chat.tsx
'use client';

import { useChat } from 'ai/react';
import { useState, useRef, useEffect } from 'react';
import { Send, Loader2, Bot, User, Code, ExternalLink } from 'lucide-react';

interface ChatProps {
  initialMessages?: Array<{ role: 'user' | 'assistant'; content: string }>;
  context?: string; // Additional context about current page
}

export function ChatInterface({ initialMessages = [], context }: ChatProps) {
  const { messages, input, handleInputChange, handleSubmit, isLoading, error, reload } = useChat({
    api: '/api/chat',
    initialMessages,
    body: {
      context, // Pass page context to AI
    },
    onError: (err) => {
      console.error('Chat error:', err);
    },
  });

  const messagesEndRef = useRef<HTMLDivElement>(null);
  const [showTools, setShowTools] = useState(true);

  // Auto-scroll to bottom
  useEffect(() => {
    messagesEndRef.current?.scrollIntoView({ behavior: 'smooth' });
  }, [messages]);

  return (
    <div className="flex flex-col h-[600px] bg-white rounded-lg border shadow-sm">
      {/* Header */}
      <div className="p-4 border-b flex items-center justify-between bg-gray-50 rounded-t-lg">
        <div className="flex items-center gap-2">
          <Bot className="w-5 h-5 text-blue-600" />
          <h3 className="font-semibold">AI Assistant</h3>
        </div>
        <div className="flex items-center gap-2 text-sm text-gray-500">
          <span className={`w-2 h-2 rounded-full ${isLoading ? 'bg-yellow-400 animate-pulse' : 'bg-green-400'}`} />
          {isLoading ? 'Thinking...' : 'Ready'}
        </div>
      </div>

      {/* Messages */}
      <div className="flex-1 overflow-y-auto p-4 space-y-4">
        {messages.map((message, index) => (
          <div
            key={message.id}
            className={`flex gap-3 ${message.role === 'user' ? 'flex-row-reverse' : ''}`}
          >
            <div className={`w-8 h-8 rounded-full flex items-center justify-center ${
              message.role === 'user' ? 'bg-blue-100' : 'bg-purple-100'
            }`}>
              {message.role === 'user' ? <User className="w-4 h-4" /> : <Bot className="w-4 h-4" />}
            </div>
            
            <div className={`flex-1 max-w-[80%] ${message.role === 'user' ? 'items-end' : 'items-start'}`}>
              <div className={`p-3 rounded-lg ${
                message.role === 'user' 
                  ? 'bg-blue-600 text-white' 
                  : 'bg-gray-100 text-gray-900'
              }`}>
                {message.content ? (
                  <div className="prose prose-sm max-w-none">
                    {message.content}
                  </div>
                ) : (
                  <div className="flex items-center gap-2 text-gray-400">
                    <Loader2 className="w-4 h-4 animate-spin" />
                    Thinking...
                  </div>
                )}
                
                {/* Tool Invocations */}
                {message.toolInvocations?.map((tool) => (
                  <div key={tool.toolCallId} className="mt-2 p-2 bg-black/10 rounded text-sm">
                    <div className="flex items-center gap-1 text-xs opacity-70 mb-1">
                      <Code className="w-3 h-3" />
                      Using {tool.toolName}
                    </div>
                    {tool.state === 'result' ? (
                      <div className="text-xs">
                        {tool.toolName === 'getDocumentation' && (
                          <a 
                            href={tool.result.url} 
                            target="_blank" 
                            rel="noopener noreferrer"
                            className="flex items-center gap-1 hover:underline"
                          >
                            View docs: {tool.result.title}
                            <ExternalLink className="w-3 h-3" />
                          </a>
                        )}
                      </div>
                    ) : (
                      <div className="text-xs opacity-70">Loading...</div>
                    )}
                  </div>
                ))}
              </div>
            </div>
          </div>
        ))}
        <div ref={messagesEndRef} />
      </div>

      {/* Error State */}
      {error && (
        <div className="p-3 bg-red-50 text-red-600 text-sm flex items-center justify-between">
          <span>Error: {error.message}</span>
          <button 
            onClick={() => reload()}
            className="text-red-700 underline"
          >
            Retry
          </button>
        </div>
      )}

      {/* Input */}
      <form onSubmit={handleSubmit} className="p-4 border-t">
        <div className="flex gap-2">
          <input
            value={input}
            onChange={handleInputChange}
            placeholder="Ask about Next.js..."
            className="flex-1 px-4 py-2 border rounded-lg focus:outline-none focus:ring-2 focus:ring-blue-500"
            disabled={isLoading}
          />
          <button
            type="submit"
            disabled={isLoading || !input.trim()}
            className="px-4 py-2 bg-blue-600 text-white rounded-lg disabled:opacity-50 disabled:cursor-not-allowed"
          >
            {isLoading ? <Loader2 className="w-5 h-5 animate-spin" /> : <Send className="w-5 h-5" />}
          </button>
        </div>
        <p className="mt-2 text-xs text-gray-500">
          AI may produce inaccurate information. Verify important code before using.
        </p>
      </form>
    </div>
  );
}
```

## 30.3 Streaming and Real-Time AI

Implement streaming patterns for real-time AI experiences beyond simple chat.

### Real-Time Streaming Component

```typescript
// components/ai/streaming-completion.tsx
'use client';

import { useState, useCallback } from 'react';
import { readStreamableValue } from 'ai/rsc';
import { streamContent } from '@/app/actions/generate';

interface StreamingCompletionProps {
  prompt: string;
  onComplete?: (content: string) => void;
}

export function StreamingCompletion({ prompt, onComplete }: StreamingCompletionProps) {
  const [content, setContent] = useState('');
  const [isStreaming, setIsStreaming] = useState(false);
  const [error, setError] = useState<string | null>(null);

  const generate = useCallback(async () => {
    setIsStreaming(true);
    setContent('');
    setError(null);

    try {
      // Server Action returns a streamable value
      const { output } = await streamContent(prompt);
      
      for await (const delta of readStreamableValue(output)) {
        if (delta) {
          setContent((prev) => prev + delta);
        }
      }
      
      onComplete?.(content);
    } catch (err) {
      setError(err instanceof Error ? err.message : 'Unknown error');
    } finally {
      setIsStreaming(false);
    }
  }, [prompt, content, onComplete]);

  return (
    <div className="space-y-4">
      <button
        onClick={generate}
        disabled={isStreaming}
        className="px-4 py-2 bg-purple-600 text-white rounded-lg disabled:opacity-50"
      >
        {isStreaming ? 'Generating...' : 'Generate Content'}
      </button>

      {content && (
        <div className="p-4 bg-gray-50 rounded-lg border">
          <div className="prose prose-sm max-w-none">
            {content}
          </div>
        </div>
      )}

      {error && (
        <div className="p-3 bg-red-50 text-red-600 rounded text-sm">
          {error}
        </div>
      )}
    </div>
  );
}

// Server Action implementation
// app/actions/generate.ts
'use server';

import { createStreamableValue } from 'ai/rsc';
import { streamText } from 'ai';
import { models } from '@/lib/ai/providers';

export async function streamContent(prompt: string) {
  const stream = createStreamableValue('');

  (async () => {
    const { textStream } = await streamText({
      model: models.fast,
      prompt,
    });

    for await (const delta of textStream) {
      stream.update(delta);
    }

    stream.done();
  })();

  return { output: stream.value };
}
```

## 30.4 AI Integration Patterns

Implement common AI patterns like RAG (Retrieval Augmented Generation) and multi-step agents.

### RAG Implementation

```typescript
// lib/ai/rag.ts
import { embed, embedMany } from 'ai';
import { models } from './providers';
import { cosineSimilarity } from '@/lib/vector-math';

interface Document {
  id: string;
  content: string;
  metadata: Record<string, any>;
  embedding?: number[];
}

export async function generateEmbeddings(texts: string[]): Promise<number[][]> {
  const { embeddings } = await embedMany({
    model: models.fast,
    values: texts,
  });
  return embeddings;
}

export async function findRelevantDocuments(
  query: string,
  documents: Document[],
  topK: number = 3
): Promise<Document[]> {
  // Generate embedding for query
  const { embedding: queryEmbedding } = await embed({
    model: models.fast,
    value: query,
  });

  // Calculate similarities
  const scored = documents.map((doc) => ({
    ...doc,
    score: doc.embedding 
      ? cosineSimilarity(queryEmbedding, doc.embedding)
      : 0,
  }));

  // Sort by relevance and return top K
  return scored
    .sort((a, b) => b.score - a.score)
    .slice(0, topK)
    .map(({ score, ...doc }) => doc);
}

// Route Handler with RAG
// app/api/chat/rag/route.ts
import { streamText, convertToCoreMessages } from 'ai';
import { findRelevantDocuments } from '@/lib/ai/rag';
import { getAllDocuments } from '@/lib/docs';

export async function POST(req: Request) {
  const { messages } = await req.json();
  const lastMessage = messages[messages.length - 1];

  // Retrieve relevant context
  const documents = await getAllDocuments();
  const relevant = await findRelevantDocuments(lastMessage.content, documents, 3);
  
  const context = relevant
    .map((doc) => `Source: ${doc.metadata.title}\n${doc.content}`)
    .join('\n\n---\n\n');

  const result = await streamText({
    model: models.balanced,
    system: `You are a helpful assistant. Use the following context to answer the question.
      If the context doesn't contain the answer, say so.
      
      Context:
      ${context}`,
    messages: convertToCoreMessages(messages),
  });

  return result.toDataStreamResponse();
}
```

### Multi-Step Agent Pattern

```typescript
// lib/ai/agent.ts
import { generateText, tool } from 'ai';
import { z } from 'zod';
import { models } from './providers';

export async function runResearchAgent(topic: string) {
  const steps: string[] = [];
  
  const result = await generateText({
    model: models.powerful,
    tools: {
      searchWeb: tool({
        description: 'Search the web for current information',
        parameters: z.object({ query: z.string() }),
        execute: async ({ query }) => {
          steps.push(`Searching web: ${query}`);
          return await searchWeb(query);
        },
      }),
      calculate: tool({
        description: 'Perform calculations',
        parameters: z.object({ expression: z.string() }),
        execute: async ({ expression }) => {
          steps.push(`Calculating: ${expression}`);
          return eval(expression); // Use safe math library in production
        },
      }),
      saveToDatabase: tool({
        description: 'Save research results',
        parameters: z.object({ 
          title: z.string(), 
          content: z.string(),
          category: z.string() 
        }),
        execute: async (data) => {
          steps.push(`Saving to DB: ${data.title}`);
          return await db.research.create({ data });
        },
      }),
    },
    system: `You are a research assistant. Given a topic:
      1. Search for current information
      2. Analyze and synthesize findings
      3. Calculate any relevant metrics
      4. Save the final report to the database`,
    prompt: `Research the following topic and create a comprehensive report: ${topic}`,
    maxSteps: 5, // Limit tool call iterations
  });

  return {
    text: result.text,
    steps,
    toolCalls: result.toolCalls,
    toolResults: result.toolResults,
  };
}
```

## 30.5 Cost Optimization and Monitoring

Manage AI costs through caching, model selection, and usage tracking.

### Intelligent Caching

```typescript
// lib/ai/cached-generation.ts
import { generateText, GenerateTextResult } from 'ai';
import { redisCache } from '@/lib/redis/cache';
import { createHash } from 'crypto';

interface CacheConfig {
  ttl: number; // seconds
  keyPrefix: string;
}

export async function cachedGenerate(
  params: Parameters<typeof generateText>[0],
  cacheConfig: CacheConfig
): Promise<GenerateTextResult> {
  // Create cache key from prompt and parameters
  const keyData = JSON.stringify({
    model: params.model,
    prompt: params.prompt,
    system: params.system,
    temperature: params.temperature,
  });
  
  const cacheKey = `${cacheConfig.keyPrefix}:${createHash('sha256').update(keyData).digest('hex')}`;

  // Check cache
  const cached = await redisCache.get<GenerateTextResult>(cacheKey);
  if (cached) {
    console.log('AI cache hit:', cacheKey);
    return cached;
  }

  // Generate fresh
  const result = await generateText(params);
  
  // Cache result (excluding raw response to save space)
  const cacheable = {
    text: result.text,
    toolCalls: result.toolCalls,
    toolResults: result.toolResults,
    finishReason: result.finishReason,
    usage: result.usage,
  };
  
  await redisCache.set(cacheKey, cacheable, cacheConfig.ttl);
  
  return result;
}

// Usage with semantic caching (similar queries return cached result)
export async function semanticCachedGenerate(
  query: string,
  params: Omit<Parameters<typeof generateText>[0], 'prompt'>
) {
  // Generate embedding for query
  const { embedding } = await embed({
    model: models.fast,
    value: query,
  });

  // Search for similar cached queries
  const similar = await findSimilarQueries(embedding);
  if (similar && similar.similarity > 0.95) {
    return similar.result;
  }

  // Generate and cache with embedding
  const result = await generateText({ ...params, prompt: query });
  await cacheWithEmbedding(query, embedding, result);
  
  return result;
}
```

### Usage Tracking and Rate Limiting

```typescript
// lib/ai/usage-tracker.ts
import { redis } from '@/lib/redis';

interface UsageMetrics {
  userId: string;
  model: string;
  promptTokens: number;
  completionTokens: number;
  totalTokens: number;
  cost: number;
  timestamp: Date;
}

export async function trackUsage(metrics: UsageMetrics) {
  const pipeline = redis.pipeline();
  
  // Increment daily usage
  const dayKey = `usage:daily:${new Date().toISOString().split('T')[0]}:${metrics.userId}`;
  pipeline.hincrby(dayKey, 'requests', 1);
  pipeline.hincrby(dayKey, 'tokens', metrics.totalTokens);
  pipeline.expire(dayKey, 60 * 60 * 24 * 30); // Keep for 30 days
  
  // Track by model
  pipeline.hincrby(`usage:models:${metrics.userId}`, metrics.model, metrics.totalTokens);
  
  // Track cost (approximate)
  const cost = calculateCost(metrics.model, metrics.promptTokens, metrics.completionTokens);
  pipeline.incrbyfloat(`usage:cost:${metrics.userId}`, cost);
  
  await pipeline.exec();
}

export async function checkUsageLimit(userId: string): Promise<{
  allowed: boolean;
  remaining: number;
  resetTime: number;
}> {
  const today = new Date().toISOString().split('T')[0];
  const usage = await redis.hgetall(`usage:daily:${today}:${userId}`);
  
  const requests = parseInt(usage.requests || '0');
  const limit = 100; // Free tier: 100 requests/day
  
  return {
    allowed: requests < limit,
    remaining: Math.max(0, limit - requests),
    resetTime: new Date(`${today}T23:59:59Z`).getTime(),
  };
}

function calculateCost(model: string, prompt: number, completion: number): number {
  const rates: Record<string, { prompt: number; completion: number }> = {
    'gpt-4o': { prompt: 5 / 1000000, completion: 15 / 1000000 }, // $5/$15 per 1M tokens
    'gpt-4o-mini': { prompt: 0.15 / 1000000, completion: 0.6 / 1000000 },
    'claude-3-5-sonnet': { prompt: 3 / 1000000, completion: 15 / 1000000 },
  };
  
  const rate = rates[model] || rates['gpt-4o'];
  return (prompt * rate.prompt) + (completion * rate.completion);
}

// Middleware to check limits
// app/api/chat/route.ts
export async function POST(req: Request) {
  const userId = await getUserId(req);
  const limit = await checkUsageLimit(userId);
  
  if (!limit.allowed) {
    return Response.json(
      { error: 'Daily limit exceeded', resetTime: limit.resetTime },
      { status: 429 }
    );
  }
  
  // ... proceed with generation
}
```

## 30.6 Privacy and Security Considerations

Handle sensitive data and ensure compliance when using AI services.

### Data Sanitization

```typescript
// lib/ai/privacy.ts
import { PiiDetector } from '@/lib/pii-detector';

export function sanitizeInput(text: string): {
  sanitized: string;
  detected: string[];
} {
  const pii = new PiiDetector();
  const detected: string[] = [];
  
  let sanitized = text
    // Remove email addresses
    .replace(/\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b/g, (match) => {
      detected.push('email');
      return '[EMAIL]';
    })
    // Remove phone numbers
    .replace(/\b\d{3}[-.]?\d{3}[-.]?\d{4}\b/g, (match) => {
      detected.push('phone');
      return '[PHONE]';
    })
    // Remove API keys
    .replace(/\b(sk-[a-zA-Z0-9]{48})\b/g, (match) => {
      detected.push('api_key');
      return '[API_KEY]';
    });
  
  return { sanitized, detected };
}

// app/api/chat/route.ts
export async function POST(req: Request) {
  const { messages } = await req.json();
  
  // Sanitize all user messages
  const sanitizedMessages = messages.map((msg: any) => {
    if (msg.role === 'user') {
      const { sanitized, detected } = sanitizeInput(msg.content);
      if (detected.length > 0) {
        console.warn('PII detected:', detected);
      }
      return { ...msg, content: sanitized };
    }
    return msg;
  });
  
  // Proceed with sanitized content
  const result = await streamText({
    // ... config
    messages: sanitizedMessages,
  });
  
  return result.toDataStreamResponse();
}
```

### Local/Edge AI for Sensitive Data

```typescript
// lib/ai/edge-model.ts
import { HfInference } from '@huggingface/inference';
import { pipeline } from '@xenova/transformers';

// Use smaller models at the edge for privacy-sensitive tasks
export async function classifyTextLocal(text: string) {
  // Runs entirely in the edge runtime, no data leaves the server
  const classifier = await pipeline(
    'text-classification',
    'Xenova/distilbert-base-uncased-finetuned-sst-2-english'
  );
  
  const result = await classifier(text);
  return result;
}

// Route Handler with local inference
// app/api/classify/route.ts
export const runtime = 'edge';

export async function POST(req: Request) {
  const { text } = await req.json();
  
  // Process locally - no external API call
  const classification = await classifyTextLocal(text);
  
  return Response.json(classification);
}
```

## Key Takeaways from Chapter 30

1. **AI SDK Configuration**: Use the Vercel AI SDK for unified provider access (OpenAI, Anthropic, Google) with consistent streaming interfaces. Configure multiple model tiers (fast/balanced/powerful) to optimize for latency vs. quality.

2. **Streaming Architecture**: Implement `streamText` for real-time responses and `createStreamableValue` for Server Actions. Use `readStreamableValue` on the client to consume streams incrementally, updating UI as tokens arrive.

3. **Tool Augmentation**: Define tools with Zod schemas to allow AI to call functions, retrieve documentation, or query databases. Use `maxSteps` to limit recursion and prevent runaway tool calling.

4. **RAG Implementation**: Generate embeddings for documents and queries using `embed()` and `embedMany()`. Use vector similarity (cosine distance) to retrieve relevant context before generation, reducing hallucinations.

5. **Cost Management**: Cache frequent queries using semantic similarity or exact hash matching. Track token usage per user with Redis, implementing rate limits and tiered access. Use cheaper models (GPT-4o-mini) for simple tasks, reserving expensive models for complex reasoning.

6. **Privacy Protection**: Sanitize PII (emails, phone numbers, API keys) before sending to external AI providers. Use local models via Transformers.js or Edge runtime for sensitive data processing. Implement data retention policies and avoid logging full prompts containing personal information.

## Coming Up Next

**Chapter 31: Internationalization (i18n)**

With AI capabilities integrated into your application, it's time to make your content accessible globally. In Chapter 31, we'll explore setting up internationalization with Next.js i18n routing, locale detection strategies, content localization workflows, RTL (Right-to-Left) language support, and SEO optimization for multi-language sites. You'll learn how to structure your application for global audiences while maintaining performance and type safety across different locales.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='29. progressive_web_applications.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../6. Specialized_topics/31. internationalization.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
